In [8]:
#Packages
using EzXML
using DataFrames
using CSV
using CSVFiles

In [21]:

cd("C:\\Users\\migue\\Documents\\You-i Lab\\SenSky\\EncuestasDinamicas")
Surveys=filter!(s->occursin(r".xml", s),readdir())



3-element Array{String,1}:
 "13preguntas.xml"
 "18preguntas.xml"
 "24preguntas.xml"

In [22]:
#Determines the maximum size of the array
global size=0
for doc in Surveys
    survey = root(readxml("$doc"))
    global v=Int[] 
    # Iterate over child elements.
    for topic in eachelement(survey)
        # Get an attribute value by name.
        topics = topic["name"]   
        for Nquestion in eachelement(topic)  
        push!(v, parse(Int, Nquestion["id"]))   
        end
        if length(v)>size
            size=length(v)
        end  
    end
end
println(size)

24


In [23]:
for document in Surveys
    survey = root(readxml("$document"))
    AQ=AA=nombres=fill("nothing", size+5)
####### Iterate over child elements.
    for topic in eachelement(survey)
####### Get an attribute value by name.
        topics = topic["name"] 
        for Nquestion in eachelement(topic)
        AQ[parse(Int, Nquestion["id"])]=nodecontent(firstnode(Nquestion))
        AA[parse(Int, Nquestion["id"])]=nodecontent(findfirst("answer", Nquestion)) 
        AA[parse(Int, Nquestion["id"]) + 5]=AA[parse(Int, Nquestion["id"])]
        end
    end
    AA[1]= document
    AA[2]= "latitude"
    AA[3]= "longitude"
    AA[4]= "workshop-key"
    AA[5]= "survey-key"
########  ARRAY TO STRING ARRAY
        AA=string.(AA)
########  ARRAY OF NAME
       for i in 6:length(AA)
        nombres[i] = "Q$(i-5)"  
        end
        nombres[1]="file-path"
        nombres[2]="latitude"
        nombres[3]="longitude"
        nombres[4]="workshop"
        nombres[5]="survey"

    pf="C:\\Users\\migue\\Documents\\You-i Lab\\SenSky\\surveyDinamica.csv"   
    if isfile(pf) #VERIFY IF THE FILE ALREADY EXISTS
        df=string.(DataFrame(load(pf)))
########check if the size of the df is different 
        if length(df[1,:])<length(AA)
            for j in length(df[1,:])+1:length(AA)
            insertcols!(df, j, nombres[j] => "nothing")
            end    
        end
        if length(df[1,:])>length(AA)
            difference  = length(df[1,:])- length(AA)
            for j in length(AA)+1:length(df[1,:])
            AA[j]= "nothing"
            end    
        end
        
########VERIFY IF THE SURVEY ALREADY EXISTS    
        if document ∉ df[:,1] 
            push!(df, AA)
            CSV.write(pf, df)
        end
    else  #CREATE FILE 
        df = DataFrame()
        for j in 1:length(AA)
        insertcols!(df, j, nombres[j] => AA[j])
        end    
        CSV.write(pf, df)
    end  
    
end
println("Listo :)")

Listo :)
